In [ ]:
# import pandas as pd
# import numpy as np
# import plotly.express as px
# import plotly.io as pio

# pio.renderers.default = "notebook"  # 如果无效，试试 "notebook_connected" 或 "iframe_connected"
# from datetime import datetime, timedelta
# # 文件列表
# files = [
#     'oxfun.csv',
#     'bybit.csv',
#     'okx.csv',
#     'ascendex.csv',
#     'cryptocom.csv',
#     'bingx.csv',
#     'lbank.csv',
#     'blofin.csv',
#     'digifinex.csv',
#     'gateio.csv',
#     'krakenfutures.csv',
#     'binance.csv',
#     'mexc.csv',
# ]

# # 获取当前时间
# now = pd.Timestamp.now()
# # 半小时前的时间点
# half_hour_ago = now - timedelta(minutes=5)

# # 合并过滤后的 DataFrame 列表
# df_list = []

# for f in files:
#     exchange = f.replace('.csv', '')
#     df = pd.read_csv("./snapshots/" + f)

#     # 转换时间为 datetime（要在这里做，才能用过滤）
#     df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

#     # 过滤条件：bid/ask > 0 且时间在半小时内
#     df = df[(df['bid'] > 0) & (df['ask'] > 0) & (df['timestamp'] >= half_hour_ago)]

#     df['exchange'] = exchange
#     df_list.append(df)

# # 合并所有 DataFrame
# df_all = pd.concat(df_list, ignore_index=True)

# # ✅ 确保 timestamp 列存在并是 datetime 类型（冗余安全检查）
# df_all['timestamp'] = pd.to_datetime(df_all['timestamp'], errors='coerce')


In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import qualitative
import os
# import kaleido
df_all = pd.read_csv('../csv_orderbooks_symbol/ob_AEVO_USDT_USDT.csv')
# 保留最近半小时的数据（可选）
latest_ts = df_all['timestamp'].max()
df_all = df_all[df_all['timestamp'] >= latest_ts - pd.Timedelta(minutes=30)]

# 筛选 XRP-USDT（你可改为 BTC-USDT）
symbol = "XRP-USDT"
df_symbol = df_all[df_all.symbol == symbol].copy()
df_symbol = df_symbol.sort_values(by=["timestamp", "exchange"])
df_symbol['timestamp_sec'] = df_symbol['timestamp'].dt.floor('s')

# === 套利计算 ===
# 找每秒 min ask
idx_min_ask = df_symbol.groupby('timestamp_sec')['ask'].idxmin()
min_ask_df = df_symbol.loc[idx_min_ask, ['timestamp_sec', 'ask', 'exchange']]
min_ask_df = min_ask_df.rename(columns={'ask': 'min_ask', 'exchange': 'min_ask_ex'})

# 找每秒 max bid
idx_max_bid = df_symbol.groupby('timestamp_sec')['bid'].idxmax()
max_bid_df = df_symbol.loc[idx_max_bid, ['timestamp_sec', 'bid', 'exchange']]
max_bid_df = max_bid_df.rename(columns={'bid': 'max_bid', 'exchange': 'max_bid_ex'})

# 合并套利信息
df_spread = pd.merge(min_ask_df, max_bid_df, on='timestamp_sec')
df_spread['spread_pct'] = (df_spread['max_bid'] - df_spread['min_ask']) / df_spread['min_ask'] * 100

# === 绘图 ===
color_palette = qualitative.Plotly
exchanges = df_symbol['exchange'].unique()
color_map = {exch: color_palette[i % len(color_palette)] for i, exch in enumerate(exchanges)}

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3],
    subplot_titles=[f"{symbol} Bid/Ask Prices by Exchange", "Taker-Taker Arbitrage (%)"]
)

# 主图：每个交易所 bid/ask
for exch in exchanges:
    df_sub = df_symbol[df_symbol.exchange == exch]

    fig.add_trace(go.Scatter(
        x=df_sub['timestamp'],
        y=df_sub['ask'],
        mode='lines+markers',
        name=f"{exch} ask",
        line=dict(color=color_map[exch], dash='solid'),
        marker=dict(size=6, opacity=0.5),
        opacity=0.5,
        hovertemplate=f"{exch} ask<br>%{{x}}<br>%{{y}}<extra></extra>"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df_sub['timestamp'],
        y=df_sub['bid'],
        mode='lines',
        name=f"{exch} bid",
        line=dict(color=color_map[exch], dash='dash'),
        opacity=1.0,
        hovertemplate=f"{exch} bid<br>%{{x}}<br>%{{y}}<extra></extra>"
    ), row=1, col=1)

# 子图：套利百分比 + 路径提示
fig.add_trace(go.Scatter(
    x=df_spread['timestamp_sec'],
    y=df_spread['spread_pct'],
    mode='lines+markers',
    name='Arbitrage (%)',
    line=dict(color='red'),
    marker=dict(size=6, opacity=0.8),
    hovertemplate=(
        "Time: %{x}<br>" +
        "Arb %: %{y:.4f}%<br>" +
        "Buy: " + df_spread['min_ask_ex'] + " @ " + df_spread['min_ask'].round(5).astype(str) + "<br>" +
        "Sell: " + df_spread['max_bid_ex'] + " @ " + df_spread['max_bid'].round(5).astype(str) +
        "<extra></extra>"
    )
), row=2, col=1)

# 布局
fig.update_layout(
    width=1328,
    height=720,
    hovermode='x unified',
    title=f"{symbol} Taker-Taker Arbitrage and Orderbook Spread",
    xaxis=dict(title="Time", rangeslider=dict(visible=False)),
    xaxis2=dict(title="Time"),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="Arbitrage (%)"),
)
fig.update_xaxes(tickformat="%H:%M:%S")

# 时间标准化 & 排序
# df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.floor('s')
# df = df.sort_values(by='timestamp')

# x轴设置（图表部分）
fig.update_xaxes(
    tickformat="%H:%M:%S",
    tickangle=45,
    title="Time (HH:MM:SS)",
    tickmode="auto"
)


# === 保存图表 ===
os.makedirs("output", exist_ok=True)

# 保存为 HTML（交互图）
html_path = "output/arbitrage_chart.html"
fig.write_html(html_path)
print(f"✅ 图表已保存为 HTML: {html_path}")




TypeError: unsupported operand type(s) for -: 'numpy.int64' and 'Timedelta'